# Computational Economics and Nonlinear Modeling (1)


##  General Introduction

PSE Summer School, 2017

Pablo Winant

## What is Julia

- developped at MIT on top of opensource technologies
    - linux / git / llvm
- syntax inspired by Matlab but:
    - more consistent
    - lots of features from high level languages
- everything is JIT-compiled
    - interpreted vs compiled treadeoff
    - -> very fast
    - most of the base library is written in Julia
- opensource/free + vibrant community


Some useful links from QuantEcon:

* [Julia cheatsheet](https://cheatsheets.quantecon.org/julia-cheatsheet.html)
* [Julia-Matlab comparison](https://cheatsheets.quantecon.org/index.html)
* [Julia essentials](https://lectures.quantecon.org/jl/julia_essentials.html)
* [Vectors, arrays and matrices](https://lectures.quantecon.org/jl/julia_arrays.html)

### an example of what you shouldn't do in Matlab

How I learnt: interpreted code is slow, so vectorize your coe.

In [2]:
function stupid_loop(I,J,K)
    t = 0.0
    for i=1:I
        for j=1:J
            for k = 1:K
                t += 1.0
            end        
        end
    end
    return t
end
@time [ stupid_loop(1000,1000,i) for i =1:10]

  0.130888 seconds (12.86 k allocations: 724.900 KiB)


10-element Array{Float64,1}:
 1.0e6
 2.0e6
 3.0e6
 4.0e6
 5.0e6
 6.0e6
 7.0e6
 8.0e6
 9.0e6
 1.0e7

Code is translated to LLVM code then to instructions for the processor. Note that processor instructions are shorter than LLVM code.

In [3]:
@code_llvm stupid_loop(10,10,10)


define double @julia_stupid_loop_60873(i64, i64, i64) #0 !dbg !5 {
top:
  %3 = icmp slt i64 %0, 1
  br i1 %3, label %L34, label %if.lr.ph

if.lr.ph:                                         ; preds = %top
  %4 = icmp slt i64 %1, 1
  %5 = icmp sgt i64 %2, 0
  %6 = select i1 %5, i64 %2, i64 0
  %7 = icmp eq i64 %6, 0
  %brmerge = or i1 %4, %7
  br i1 %brmerge, label %L34, label %if3.lr.ph.if3.lr.ph.split_crit_edge.preheader

if3.lr.ph.if3.lr.ph.split_crit_edge.preheader:    ; preds = %if.lr.ph
  br label %if3.lr.ph.if3.lr.ph.split_crit_edge

L5.loopexit:                                      ; preds = %L13.loopexit
  %8 = add i64 %"#temp#2.013", 1
  %9 = icmp eq i64 %"#temp#2.013", %0
  br i1 %9, label %L34.loopexit, label %if3.lr.ph.if3.lr.ph.split_crit_edge

if3.lr.ph.if3.lr.ph.split_crit_edge:              ; preds = %if3.lr.ph.if3.lr.ph.split_crit_edge.preheader, %L5.loopexit
  %t.014 = phi double [ %13, %L5.loopexit ], [ 0.000000e+00, %if3.lr.ph.if3.lr.ph.split_crit_edge.preheader ]
 

In [4]:
@code_native stupid_loop(10,10,10)

	.text
Filename: In[2]
	pushq	%rbp
	movq	%rsp, %rbp
	xorpd	%xmm0, %xmm0
Source line: 3
	testq	%rdi, %rdi
	jle	L123
	xorl	%ecx, %ecx
Source line: 5
	testq	%rdx, %rdx
	cmovnsq	%rdx, %rcx
Source line: 4
	testq	%rsi, %rsi
	jle	L123
	testq	%rcx, %rcx
	je	L123
	xorpd	%xmm0, %xmm0
	movl	$1, %r8d
	movabsq	$140536849078656, %rax  # imm = 0x7FD148F79980
	movsd	(%rax), %xmm1           # xmm1 = mem[0],zero
	movl	$1, %edx
	jmp	L105
L63:
	movl	$1, %edx
	jmp	L105
Source line: 3
L70:
	cmpq	%rdi, %r8
	leaq	1(%r8), %r8
	jne	L63
	jmp	L123
	nopw	%cs:(%rax,%rax)
Source line: 4
L96:
	cmpq	%rsi, %rdx
	leaq	1(%rdx), %rdx
	je	L70
L105:
	movq	%rcx, %rax
	nopl	(%rax)
Source line: 6
L112:
	addsd	%xmm1, %xmm0
Source line: 5
	decq	%rax
	jne	L112
	jmp	L96
Source line: 10
L123:
	popq	%rbp
	retq
	nopl	(%rax)


## Syntax Review

### Variable assignment

Assignement operator is = (equality is ==, identity is ===)

In [5]:
# Assign the value 10 to the variable x
x = 10

10

In [6]:
2 == 3

false

In [7]:
# Variable names can have Unicode characters
# To get ϵ in the REPL, type \epsilon<TAB>
ϵ = 1e-4

0.0001

Default semantic is pass-by-reference:

In [8]:
a = [1,2,3,4]
b = a
a[1] = 10
b

4-element Array{Int64,1}:
 10
  2
  3
  4

To work on a copy: `copy` or `deepcopy`

In [9]:
a = [1,2,3,4]
b = copy(a)
a[1]=10
b

4-element Array{Int64,1}:
 1
 2
 3
 4

In [10]:
a .== b

4-element BitArray{1}:
 false
  true
  true
  true

In [11]:
a === b

false

### Basic types

In [12]:
# for any object `typeof` returns the type
?typeof

LoadError: [91mUndefVarError: ? not defined[39m

In [13]:
typeof(a)

Array{Int64,1}

#### Numbers

In [14]:
y = 2 + 2

4

In [15]:
-y

-4

In [16]:
0.34*23

7.82

In [17]:
3/4

0.75

In [30]:
# Scalar multiplication doesn't require *
3(4 - 2)

6

In [18]:
x = 4
2x

8

In [19]:
typeof(x)

Int64

In [21]:
sizeof(a)

32

#### Booleans

Equality

In [22]:
0 == 1

false

In [36]:
2 != 3

true

In [37]:
3 <= 4

true

Identity

In [24]:
a = [34, 35]
b = [34, 35]
c = a

2-element Array{Int64,1}:
 34
 35

In [43]:
c === a

true

In [42]:
b === a

false

Boolean operator

In [25]:
true && false

false

In [26]:
true || false

true

In [44]:
!true

false

#### Strings

In [46]:
# Strings are written using double quotes
str = "This is a string"

"This is a string"

In [45]:
ch = 'k' # this is a character

'k': ASCII/Unicode U+006b (category Ll: Letter, lowercase)

In [47]:
# Strings can also contain Unicode characters
fancy_str = "α is a string"

"α is a string"

In [28]:
# String interpolation using $
# The expression in parentheses is evaluated and the result is 
# inserted into the string
a = 2+2
"2 + 2 = $(a)"

"2 + 2 = 4"

In [29]:
println("It took me $(a) iterations")

It took me 4 iterations


In [30]:
# String concatenation using *
"hello" * "world"

"helloworld"

In [31]:
println("hello ", "world")

hello world


#### Arrays

Julia has one-dimensional arrays. They are also called Vector.

In [33]:
A = [1, 2]

2-element Array{Int64,1}:
 1
 2

In [34]:
typeof(A) == Vector{Int64}

true

In [38]:
A''

2-element Array{Int64,1}:
 1
 2

2d arrays are also called matrices... and can be used for matrix multiplications.

In [42]:
a1 = [1,2,3,4]
a2 = [1,2,3,4]+4
[a1; a2]
cat(1, a1, a2)

8-element Array{Int64,1}:
 1
 2
 3
 4
 5
 6
 7
 8

In [44]:
b = [1 0.6 0]

1×3 Array{Float64,2}:
 1.0  0.6  0.0

In [43]:
B = [0.1 0.2 0.3; 4 5 6]

2×3 Array{Float64,2}:
 0.1  0.2  0.3
 4.0  5.0  6.0

In [47]:
B*B'

2×2 Array{Float64,2}:
 0.14   3.2
 3.2   77.0

Vectorized operations take a ., even comparisons:

In [63]:
B.*B .< B

2×3 BitArray{2}:
  true   true   true
 false  false  false

In [48]:
f(x) = x^2+1

f (generic function with 1 method)

In [49]:
f(43)

1850

In [51]:
f.(B)

2×3 Array{Float64,2}:
  1.01   1.04   1.09
 17.0   26.0   37.0 

Elements are always accessed with square brackets:

In [64]:
B[:,1]

2-element Array{Float64,1}:
 0.1
 4.0

In [65]:
B[:,1:end-1]

2×2 Array{Float64,2}:
 0.1  0.2
 4.0  5.0

#### Control flow

In [ ]:
Conditions

In [ ]:
x = -3
if x < 0
    println("x is negative")
elseif x > 0 # optional and unlimited
    println("x is positive")
else         # optional
    println("x is zero")
end

In [ ]:
While

In [52]:
i = 3
while i > 0
    println(i)
    i = i - 1
end

3
2
1


For loops

In [53]:
# Iterate through ranges of numbers
for i = 1:3
    println(i)
end

1
2
3


In [54]:
# Iterate through arrays
cities = ["Boston", "New York", "Philadelphia"]
for city in cities
    println(city)
end

Boston
New York
Philadelphia


In [55]:
cities

3-element Array{String,1}:
 "Boston"      
 "New York"    
 "Philadelphia"

In [58]:
states

3-element Array{String,1}:
 "MA"
 "NY"
 "PA"

In [60]:
collect( zip(cities, states) )

3-element Array{Tuple{String,String},1}:
 ("Boston", "MA")      
 ("New York", "NY")    
 ("Philadelphia", "PA")

In [61]:
# Iterate through arrays of tuples using zip
states = ["MA", "NY", "PA"]
for (city, state) in zip(cities, states)
    println("$city, $state")
end

Boston, MA
New York, NY
Philadelphia, PA


In [62]:
# Iterate through arrays and their indices using enumerate
for (i, city) in enumerate(cities)
    println("City $i is $city")
end

City 1 is Boston
City 2 is New York
City 3 is Philadelphia


#### List comprehensions

In [80]:
[i^2 for i=1:10]

10-element Array{Int64,1}:
   1
   4
   9
  16
  25
  36
  49
  64
  81
 100

In [63]:
[i^2 for i=1:10 if mod(i,2)==0]

5-element Array{Int64,1}:
   4
  16
  36
  64
 100

### Data Types and multiple dispatch

#### Composite types

A **composite type** is a collection of named fields that can be treated as a single value. They bear a passing resemblance to MATLAB structs.

All fields must be declared ahead of time. The double colon, `::`, constrains a field to contain values of a certain type. This is optional for any field.

In [64]:
# Type definition
type Parameter
    value::Float64
    transformation::Function # Function is a type!
    tex_label::String
    description::String
end

When a type with $n$ fields is defined, a constructor (function that creates an instance of that type) that takes $n$ ordered arguments is automatically created. Additional constructors can be defined for convenience.

In [65]:
# Creating an instance of the Parameter type using the default
# constructor
β = Parameter(0.9, identity, "\beta", "Discount rate")

Parameter(0.9, identity, "\beta", "Discount rate")

In [66]:
# Alternative constructors end with an appeal to the default
# constructor
function Parameter(value::Float64, tex_label::String)
    transformation = identity
    description = "No description available"
    return Parameter(value, transformation, tex_label, description)
end

α = Parameter(0.5, "\alpha")

Parameter(0.5, identity, "\alpha", "No description available")

Now the function `Parameter` has two different `methods` with different signatures:

In [67]:
methods(Parameter)

# 4 methods for generic function "(::Type)":
Parameter(value::Float64, transformation::Function, tex_label::String, description::String) in Main at In[64]:3
Parameter(value::Float64, tex_label::String) in Main at In[66]:4
Parameter(value, transformation, tex_label, description) in Main at In[64]:3
(::Type{T})(arg) where T in Base at sysimg.jl:24

In [83]:
fun(x::Int64, y::Int64) = x^3 + y

fun (generic function with 5 methods)

In [84]:
fun(x::Float64, y::Int64) = x/2 + y

fun (generic function with 5 methods)

In [85]:
fun(2, 2)

10

In [86]:
fun(2.0, 2)

3.0

In [87]:
# Find the fields of an instance of a composite type
fieldnames(α)

4-element Array{Symbol,1}:
 :value         
 :transformation
 :tex_label     
 :description   

In [88]:
α.tex_label

"\alpha"

In [93]:
# Access a particular field using .
α.value

0.75

In [92]:
# Fields are modifiable and can be assigned to, like 
# ordinary variables
α.value = 0.75

0.75

### Parameterized Types

**Parameterized types** are data types that are defined to handle values identically regardless of the type of those values.

Arrays are a familiar example. An `Array{T,1}` is a one-dimensional array filled with objects of any type `T` (e.g. `Float64`, `String`).

In [89]:
# Defining a parametric point
type Duple{T} # T is a parameter to the type Duple
    x::T
    y::T
end

This single declaration defines an unlimited number of new types: `Duple{String}`, `Duple{Float64}`, etc. are all immediately usable.

In [94]:
sizeof( Duple(3.0, -15.0) )

16

In [95]:
Duple("Broadway", "42nd St")

Duple{String}("Broadway", "42nd St")

In [96]:
# What happens here?
Duple(1.5, 3)

LoadError: [91mMethodError: no method matching Duple(::Float64, ::Int64)[0m
Closest candidates are:
  Duple(::T, [91m::T[39m) where T at In[89]:3[39m

We can also restrict the type parameter `T`:

In [97]:
# T can be any subtype of Number, but nothing else
type PlanarCoordinate{T<:Number}
    x::T
    y::T
end

In [98]:
PlanarCoordinate("4th Ave", "14th St")

LoadError: [91mMethodError: no method matching PlanarCoordinate(::String, ::String)[39m

In [99]:
PlanarCoordinate(2//3, 8//9)

PlanarCoordinate{Rational{Int64}}(2//3, 8//9)

### Why Use Types?

You can write all your code without thinking about types at all. If you do this, however, you’ll be missing out on some of the biggest benefits of using Julia.

If you understand types, you can:

- Write faster code
- Write expressive, clear, and well-structured programs (keep this in mind when we talk about functions)
- Reason more clearly about how your code works

Even if you only use built-in functions and types, your code still takes advantage of Julia’s type system. That’s why it’s important to understand what types are and how to use them.

In [100]:
# Example: writing type-stable functions
function sumofsins_unstable(n::Integer)  
    sum = 0  
    for i in 1:n  
        sum += sin(3.4)  
    end  
    return sum 
end  

function sumofsins_stable(n::Integer)  
    sum = 0.0  
    for i in 1:n  
        sum += sin(3.4)  
    end  
    return sum 
end

# Compile and run
sumofsins_unstable(Int(1e5))
sumofsins_stable(Int(1e5))

-25554.110202663698

In [103]:
@time sumofsins_unstable(Int(1e5))

  0.011648 seconds (300.00 k allocations: 4.578 MiB)


-25554.110202663698

In [104]:
@time sumofsins_stable(Int(1e5))

  0.003414 seconds (6 allocations: 192 bytes)


-25554.110202663698

In `sumofsins_stable`, the compiler is guaranteed that `sum` is of type `Float64` throughout; therefore, it saves time and memory. On the other hand, in `sumofsins_unstable`, the compiler must check the type of `sum` at each iteration of the loop. Let's look at the LLVM [intermediate representation](http://www.johnmyleswhite.com/notebook/2013/12/06/writing-type-stable-code-in-julia/).

### Multiple Dispatch

So far we have defined functions over argument lists of any type. Methods allow us to define functions “piecewise”. For any set of input arguments, we can define a **method**, a definition of one possible behavior for a function.

In [116]:
# Define one method of the function print_type
function print_type(x::Number)
    println("$x is a number")
end

print_type (generic function with 1 method)

In [117]:
# Define another method
function print_type(x::String)
    println("$x is a string")
end

print_type (generic function with 2 methods)

In [118]:
# Define yet another method
function print_type(x::Number, y::Number)
    println("$x and $y are both numbers")
end

print_type (generic function with 3 methods)

In [119]:
# See all methods for a given function
methods(print_type)

# 3 methods for generic function "print_type":
print_type(x::String) in Main at In[117]:3
print_type(x::Number) in Main at In[116]:3
print_type(x::Number, y::Number) in Main at In[118]:3

Julia uses **multiple dispatch** to decide which method of a function to execute when a function is applied. In particular, Julia compares the types of _all_ arguments to the signatures of the function’s methods in order to choose the applicable one, not just the first (hence "multiple").

In [120]:
print_type(5)

5 is a number


In [ ]:
print_type("foo")

In [ ]:
print_type([1, 2, 3])

#### Other types of functions

Julia supports a short function definition for one-liners

In [105]:
f(x::Float64) = x^2.0
f(x::Int64) = x^3

f (generic function with 3 methods)

As well as a special syntax for anonymous functions

In [106]:
u->u^2

(::#7) (generic function with 1 method)

In [107]:
map(u->u^2, [1,2,3,4])

4-element Array{Int64,1}:
  1
  4
  9
 16

### Writing Julian Code

As we've seen, you can use Julia just like you use MATLAB and get faster code. However, to write faster and _better_ code, attempt to write in a “Julian” manner:

- Define composite types as logically needed
- Write type-stable functions for best performance
- Take advantage of multiple dispatch to write code that looks like math
- Add methods to existing functions

### Just-in-Time Compilation

How is Julia so fast? Julia is just-in-time (JIT) compiled, which  means (according to [this StackExchange answer](http://stackoverflow.com/questions/95635/what-does-a-just-in-time-jit-compiler-do)):

> A JIT compiler runs after the program has started and compiles the code (usually bytecode or some kind of VM instructions) on the fly (or just-in-time, as it's called) into a form that's usually faster, typically the host CPU's native instruction set. _A JIT has access to dynamic runtime information whereas a standard compiler doesn't and can make better optimizations like inlining functions that are used frequently._

> This is in contrast to a traditional compiler that compiles all the code to machine language before the program is first run.

In particular, Julia uses type information at runtime to optimize how your code is compiled. This is why writing type-stable code makes such a difference in speed!

## Additional Exercises

Taken from QuantEcon's [Julia Essentials](https://lectures.quantecon.org/jl/julia_essentials.html) and [Vectors, Arrays, and Matrices](https://lectures.quantecon.org/jl/julia_arrays.html) lectures.

1. Consider the polynomial $$p(x) = \sum_{i=0}^n a_0 x^0$$ Using `enumerate`, write a function `p` such that `p(x, coeff)` computes the value of the polynomial with coefficients `coeff` evaluated at `x`.

2. Write a function `solve_discrete_lyapunov` that solves the discrete Lyapunov equation $$S = ASA' + \Sigma \Sigma'$$ using the iterative procedure $$S_0 = \Sigma \Sigma'$$ $$S_{t+1} = A S_t A' + \Sigma \Sigma'$$ taking in as arguments the $n \times n$ matrix $A$, the $n \times k$ matrix $\Sigma$, and a number of iterations.

In [118]:
function p(x::Float64, coeffs::Vector{Float64})
    sum([c*x^(i-1) for (i,c) in enumerate(coeffs)])
end

p (generic function with 1 method)

In [125]:
c = [1/factorial(k-1) for k=1:20];

In [126]:
p(0.3, c)

1.3498588075760032

In [127]:
exp(0.3)

1.3498588075760032